In [49]:
import pandas as pd
import numpy as np

$Y = median\_house\_value \Rightarrow$ Verkliga värden, målvariabel

$X = designmatris \Rightarrow$ Alla regressorer, shape

$\hat{Y} = \hat{\beta}_0 + \hat{\beta}_1X_1 + \hat{\beta}_2X_2 + \hat{\beta}_3X_3 ... + \hat{\beta}_dX_d \Rightarrow$ Modellens prediktion för varje observation

$e_i = Y_i - \hat{Y}_i \Rightarrow$ Residual för observation $i$

$SSE = \sum_{i=1}^{n} e_i^2 \Rightarrow$ Sum of Squared Errors - totala felet för modellen

$S_{yy} = \sum(Y_i - \bar{Y})^2 = \sum{Y_{i}}^2 - \frac{(\sum{Y_i})^2}{n} \Rightarrow$ Totala variationen i Y

$SSR = S_{yy} - SSE \Rightarrow$ Regression Sum of Squares - variation som modellen förklarar

$R^2 = \frac{SSR}{S_{yy}} \Rightarrow$ Bestämningskoefficient - hur mycket av Y som modellen förklarar

$\hat{\sigma}^2 = \frac{SSE}{n-d-1} \Rightarrow$ Estimering av residualvarians

$RMSE = \sqrt{\frac{SSE}{n-d-1}} \Rightarrow$ Root Mean Squared Error - spridning i samma enhet som Y (använder $\sqrt{\frac{SSE}{n-d-1}}$ istället för $\sqrt{\frac{SSE}{n}}$ då den senare används i större utsträckning inom ML och uppgiften skulle vara en statistisk analys)

$\hat{\beta} = (X^TX)^{-1}X^TY \Rightarrow$ OLS-koefficienter - beräkning av alla $\beta$ från datan

$C = \hat{\sigma}^2(X^TX)^{-1} \Rightarrow$ Varians och kovarians för $\hat{\beta}$, används i t-test och konfidensintervall

$t_i = \frac{\hat{\beta}_i}{\hat{\sigma}\sqrt{c_{ii}}} \Rightarrow$ t-statistik - Testar om en enskild koefficient skiljer sig signifikant från 0

$F = \frac{SSR/d}{\hat{\sigma}^2} \Rightarrow$ Testar om alla regressorer tillsammans är signifikanta

$\hat{\beta}_i \pm t_{\alpha/2, n-d-1}\hat{\sigma}\sqrt{c_{ii}} \Rightarrow$ Intervallet för $\beta$ med viss säkerhetsnivå

$r = \frac{Cov(X_a,X_b)}{\sqrt{Var(X_a)Var(X_b)}} \Rightarrow$ Pearson $r$ - Mäta linjärt beroende mellan två regressorer

$d$ = antal regressorer  
$n$ = antal observationer  
$c_{ii}$ = diagonalelement i varians-kovariansmatrisen $C$  
Alla formler bygger direkt på residualerna $e_i$ och $X$-matrisen

Datasetet läses in som en dataframe och aggregeras med funktionen aggregate från linreg.py till en observation per geografisk punkt (longitude, latitude), då flera koordinater förekommer mer än en gång. Additiva variabler summeras medan nivåvariabler ersätts av det nya medelvärdet. Den kategoriska variabeln ocean_proximity var konstant inom varje grupp vilket även verifierades och möjliggjorde oförändrad inkludering efter aggregering där första värdet per grupp valdes, detta valet påverkade dock alltså inget då alla grupper bara hade samma värden så man hade lika gärna kunnat välja vilket som av värdena inom gruppen.

Därefter one-hot-encodas dataframen med funktionen prepare_features från linreg.py där första kategorin droppas, i detta fallet <1h from ocean som var den näst mest förekommande på drygt 1/3 av grupperna, vilket även är det rimligaste valet då tre av de återstående fyra kategorierna omfattas av <1h from ocean även om de har mer specifika beskrivningar.

In [50]:
from linreg import LinearRegression, aggregate, build_X_Y
housing = pd.read_csv('housing.csv')
aggregated = aggregate(housing)

def prepare_features(df, cat_cols=None, drop_first=None):
    """
    One-hot encodar angivna kategoriska kolumner och returnerar 
    dataframe redo för regression tillsammans med lista på feature-namn.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame som ska transformerad
    cat_cols : list of str
        Kolumner som ska one-hot encodas
    drop_first : bool, default True
        Drop first category för att undvika multikollinearitet
    
    Returns
    -------
    df_prepared : pd.DataFrame
        DataFrame med en-hot encodade kolumner
    feat_cols : list of str
        Lista på kolumnnamn som ska användas som X
    """

    df_prepared = df.copy()

    if cat_cols is not None:
        df_prepared = pd.get_dummies(df_prepared, columns=cat_cols, drop_first=drop_first)
    
    feat_cols = [col for col in df_prepared.columns if col not in ['median_house_value']]
    
    return df_prepared, feat_cols

aggregated, X_cols = prepare_features(aggregated, cat_cols=['ocean_proximity'], drop_first=True)
aggregated


,longitude,latitude,housing_median_age,median_income,median_house_value,total_rooms,total_bedrooms,population,households,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-124.35,40.54,52.0,3.0147,94600.0,1820.0,300.0,806.0,270.0,False,False,False,True
1,-124.30,41.80,19.0,1.9797,85800.0,2672.0,552.0,1298.0,478.0,False,False,False,True
2,-124.30,41.84,17.0,3.0313,103600.0,2677.0,531.0,1244.0,456.0,False,False,False,True
3,-124.27,40.69,36.0,2.5179,79000.0,2349.0,528.0,1194.0,465.0,False,False,False,True
4,-124.26,40.58,52.0,2.3571,111400.0,2217.0,394.0,907.0,369.0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12585,-114.56,33.69,17.0,1.6509,85700.0,720.0,174.0,333.0,117.0,True,False,False,False
12586,-114.55,32.80,19.0,1.2750,56100.0,2570.0,820.0,1431.0,608.0,True,False,False,False
12587,-114.49,33.97,17.0,1.6154,87500.0,2809.0,635.0,83.0,45.0,True,False,False,False
12588,-114.47,34.40,19.0,1.8200,80100.0,7650.0,1901.0,1129.0,463.0,True,False,False,False


Bygger X- och Y-matriser för regression och lägger till intercept $\beta_0$

In [51]:

X, Y = build_X_Y(aggregated, X_cols)
n = X.shape[0]
d = X.shape[1] - 1


kör OLS-formeln samt beräkning av grundläggande statistik så som $\hat{\sigma}^2$ och $RMSE$

In [52]:
model = LinearRegression(X, Y)
print(f'modellens beta-värden: {model.b_est}')         # modellens beta-värden
print(f'modellens prediktioner: {model.Y_est}')        # modellens prediktioner
print(f'modellens residualer: {model.residuals}')      # residualer
print(f'modellens SSE: {model.SSE}')                   # sum of squared errors
print(f'modellens varians: {model.variance}')          # modellens varians
print(f'modellens STD: {model.STD}')                   # modellens standardavvikelse
print(f'modellens RMSE: {model.RMSE}')                 # modellens RMSE (nästan samma som denna modellens STD p.g.a. stort n-tal)

modellens beta-värden: [-1.68425122e+06 -2.02550076e+04 -1.95460964e+04  8.07678211e+02
  3.89702735e+04 -8.57665947e-01  3.47044243e+01 -2.26211482e+01
  3.64048403e+01 -4.35143715e+04  1.70054470e+05 -8.41692780e+03
  1.15252233e+04]
modellens prediktioner: [213515.43665521 125344.37620943 163615.86977736 ...  23321.2310764
  55439.32646054  23559.63880305]
modellens residualer: [-118915.43665521  -39544.37620943  -60015.86977736 ...   64178.7689236
   24660.67353946   43340.36119695]
modellens SSE: 49231258393954.016
modellens varians: 3914388041.1826363
modellens STD: 62565.07045614698
modellens RMSE: 62532.76084603433


In [53]:

print(f'modellens varians-kovariansmatris: {model.C}') # hur osäkra modellens koefficienter är
print(f'modellens t-statistik: {model.t_stats}')       # mäter signifikans för varje koefficient
print(f'modellens p-värden: {model.p_values}')         # sannolikheten att koefficienten egentligen är 0; lågt värde = signifikant

modellens varians-kovariansmatris: [[ 7.86383336e+09  8.96442512e+07  8.01462962e+07  2.85655169e+05
   3.36027209e+06 -1.77760995e+03 -4.49014455e+04  5.24223588e+02
   5.52887690e+04 -9.42145134e+07  5.71009572e+07  3.63924151e+07
   3.14151041e+07]
 [ 8.96442512e+07  1.03750106e+06  9.63710658e+05  4.20897721e+03
   4.99432199e+04 -2.36227548e+01 -4.71506256e+02  1.96070388e+01
   5.92672258e+02 -1.12505473e+06  7.43550030e+05  3.17829847e+05
   3.74185841e+05]
 [ 8.01462962e+07  9.63710658e+05  9.82745770e+05  3.75659639e+03
   5.49504120e+04 -2.52015009e+01 -3.57964530e+02  4.21434174e+01
   4.54940347e+02 -1.19589176e+06  8.56377183e+05  3.54058174e+04
   3.41530438e+05]
 [ 2.85655169e+05  4.20897721e+03  3.75659639e+03  2.71212320e+03
   2.12014834e+03  4.33463270e+00  1.08322392e+01  4.50234481e-01
  -3.09996705e+01  7.30339179e+03 -3.67364570e+04 -1.61853116e+04
   2.55415595e+02]
 [ 3.36027209e+06  4.99432199e+04  5.49504120e+04  2.12014834e+03
   1.34435014e+05 -9.22193372e+

| Index | Variabel | p-värde | %  |
|:-----:|----------|---------|----:|
| 0 | intercept | 2.51e-79 | $\lt$ 0,01% |
| 1 | longitude | 1.16e-86 | $\lt$ 0,01% |
| 2 | latitude | 2.97e-85 | $\lt$ 0,01% |
| 3 | housing_median_age | 9.48e-54 | $\lt$ 0,01% |
| 4 | median_income | 0.00e+00 | $\ll$ 0,01% |
| 5 | total_rooms | 1.03e-01 | $\approx$ 10,3% |
| 6 | total_bedrooms | 6.72e-16 | $\lt$ 0,01% |
| 7 | population | 1.50e-198 | $\lt$ 0,01% |
| 8 | households | 7.48e-13 | $\lt$ 0,01% |
| 9 | INLAND | 1.35e-115 | $\lt$ 0,01% |
| 10 | ISLAND | 1.36e-09 | $\lt$ 0,01% |
| 11 | NEAR_BAY | 4.12e-04 | $\approx$ 0,04% |
| 12 | NEAR_OCEAN | 9.30e-10 | $\lt$ 0,01% |


Detta är då signifikansen för de olika kolumnerna där vi då kan se att median income visar extrem statistisk signifikans till median house value, total rooms har runt .103 och är alltså inte något vidare signifikant och resterande har hög signifikans där near bay är en utstickare (fortfarande väldigt hög signifikans men märkbart lägre än de andra)

In [54]:
print(f'Medelvärdet av hurpriserna: {model.Y_mean}')
print(f'Modellens Syy: {model.Syy}')
print(f'Modellens SSR: {model.SSR}')

Medelvärdet av hurpriserna: 201092.05267625765
Modellens Syy: 170054148897735.6
Modellens SSR: 120822890503781.58


In [55]:
print(f'Modellens F_stat: {model.F_stat}')
print(f'Modellens p_value: {model.F_p_value}')
print(f'Modellens R2: {model.R2}')

Modellens F_stat: 2572.1962418438443
Modellens p_value: 0.0
Modellens R2: 0.7104965758667852


In [56]:
# Hämta konfidensintervall
intervals = model.confidence_intervals(alpha=0.05)  # 95% CI

# Skriv ut
for i, (low, high) in enumerate(intervals):
    print(f"Koef {i} ({X_cols[i-1] if i>0 else 'intercept'}): {low:.3f} — {high:.3f}")


Koef 0 (intercept): -1858074.138 — -1510428.300
Koef 1 (longitude): -22251.576 — -18258.439
Koef 2 (latitude): -21489.265 — -17602.928
Koef 3 (housing_median_age): 705.597 — 909.759
Koef 4 (median_income): 38251.576 — 39688.971
Koef 5 (total_rooms): -1.888 — 0.173
Koef 6 (total_bedrooms): 26.292 — 43.117
Koef 7 (population): -24.070 — -21.173
Koef 8 (households): 26.463 — 46.346
Koef 9 (ocean_proximity_INLAND): -47207.869 — -39820.874
Koef 10 (ocean_proximity_ISLAND): 115090.271 — 225018.668
Koef 11 (ocean_proximity_NEAR BAY): -13086.110 — -3747.745
Koef 12 (ocean_proximity_NEAR OCEAN): 7837.243 — 15213.204


In [57]:
num_corr = model.pearson_numeric(
    feat_names=X_cols,        # listan med namn på kolumner i X
    cat_cols=['ocean_proximity_INLAND', 'ocean_proximity_ISLAND', 
              'ocean_proximity_NEAR BAY', 'ocean_proximity_NEAR OCEAN'],  # dummies som ska ignoreras
    target_col='median_house_value'
)
num_corr['abs_r'] = num_corr['Pearson_r'].abs()
num_corr



,Var1,Var2,Pearson_r,abs_r
0,longitude,longitude,1.000000,1.000000
1,longitude,latitude,-0.910462,0.910462
2,longitude,housing_median_age,-0.097817,0.097817
3,longitude,median_income,0.017519,0.017519
4,longitude,total_rooms,0.112792,0.112792
5,longitude,total_bedrooms,0.119243,0.119243
6,longitude,population,0.148764,0.148764
7,longitude,households,0.106515,0.106515
8,longitude,median_house_value,-0.009701,0.009701
9,latitude,latitude,1.000000,1.000000


In [58]:
numeric_cols = [
    'longitude',
    'latitude',
    'housing_median_age',
    'median_income',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_house_value'
]

idx_to_name = dict(enumerate(numeric_cols))

X_full = aggregated[numeric_cols].to_numpy(dtype=float)

raw_corr = model.pearson_numeric2(X_full)

corr_df = pd.DataFrame(
    raw_corr,
    columns=['idx1', 'idx2', 'Pearson_r']
)

corr_df['Var1'] = corr_df['idx1'].map(idx_to_name)
corr_df['Var2'] = corr_df['idx2'].map(idx_to_name)

corr_df['abs_r'] = corr_df['Pearson_r'].abs()
corr_df
target_corr2 = corr_df[corr_df['Var2'] == 'median_house_value']
value_sorted2 = target_corr2.sort_values(by='abs_r', ascending=False)

value_sorted2

,idx1,idx2,Pearson_r,Var1,Var2,abs_r
44,8,8,1.000000,median_house_value,median_house_value,1.000000
29,3,8,0.747927,median_income,median_house_value,0.747927
16,1,8,-0.211825,latitude,median_house_value,0.211825
34,4,8,0.200213,total_rooms,median_house_value,0.200213
43,7,8,0.118080,households,median_house_value,0.118080
38,5,8,0.104872,total_bedrooms,median_house_value,0.104872
23,2,8,0.078236,housing_median_age,median_house_value,0.078236
41,6,8,0.038130,population,median_house_value,0.038130
8,0,8,-0.009701,longitude,median_house_value,0.009701


In [59]:
aggregated2 = aggregate(housing)


cat_dummies = pd.get_dummies(
    aggregated2['ocean_proximity'],
    drop_first=False
)

X_cat = cat_dummies.to_numpy()
cat_names = cat_dummies.columns.tolist()

X_num = aggregated[numeric_cols].to_numpy()

raw_cat_corr = model.pearson_categorical_numeric2(X_cat, X_num)

corr_cat_df = pd.DataFrame(
    raw_cat_corr,
    columns=['cat_idx', 'num_idx', 'Pearson_r']
)

cat_idx_to_name = dict(enumerate(cat_names))
num_idx_to_name = dict(enumerate(numeric_cols))

corr_cat_df['Category'] = corr_cat_df['cat_idx'].map(cat_idx_to_name)
corr_cat_df['Variable'] = corr_cat_df['num_idx'].map(num_idx_to_name)


corr_cat_df.sort_values(
    by=['Pearson_r'],
    key=abs,
    ascending=[False]
)


,cat_idx,num_idx,Pearson_r,Category,Variable
17,1,8,-0.551185,INLAND,median_house_value
1,0,1,-0.393069,<1H OCEAN,latitude
27,3,0,-0.377290,NEAR BAY,longitude
10,1,1,0.343733,INLAND,latitude
12,1,3,-0.336221,INLAND,median_income
8,0,8,0.330131,<1H OCEAN,median_house_value
16,1,7,-0.263862,INLAND,households
28,3,1,0.262329,NEAR BAY,latitude
15,1,6,-0.251180,INLAND,population
3,0,3,0.241429,<1H OCEAN,median_income
